In [1]:
# import questions.csv
import pandas as pd


questions = pd.read_csv('../data/questions.csv')
questions.head(5)

,question,qnum,person,wikidataperson,text,date
0,https://id.parliament.uk/0IwOO533,903422,https://id.parliament.uk/Vs3bGLNz,http://www.wikidata.org/entity/Q689287,Which three departments had the lowest proport...,2023-01-05+00:00
1,https://id.parliament.uk/dRJ9254g,117748,https://id.parliament.uk/eD0yd5Ec,http://www.wikidata.org/entity/Q19871931,"To ask the Secretary of State for Foreign, Com...",2023-01-06+00:00
2,https://id.parliament.uk/e4IDRkKi,117410,https://id.parliament.uk/RlIqlixq,http://www.wikidata.org/entity/Q695228,"To ask the Secretary of State for Foreign, Com...",2023-01-06+00:00
3,https://id.parliament.uk/rXZ0r1JW,117477,https://id.parliament.uk/1DzMUkBy,http://www.wikidata.org/entity/Q337812,"To ask the Secretary of State for Foreign, Com...",2023-01-06+00:00
4,https://id.parliament.uk/SmC4cGhg,117777,https://id.parliament.uk/tOfis7j9,http://www.wikidata.org/entity/Q27942399,To ask the Secretary of State for Health and S...,2023-01-06+00:00


In [2]:
import geopandas as gpd
en = gpd.read_file("../shape/english_region_region.shp")


sct_wls = gpd.read_file("../shape/scotland_and_wales_region.shp")
#sct_wls = sct_wls.drop_duplicates(subset=['NAME'])

ni = gpd.read_file("../shape/OSNI_Open_Data_-_Largescale_Boundaries_-_County_Boundaries_.shp")

ni.rename(columns={'CountyName': 'NAME'}, inplace=True)
ni["NAME"] = ni["NAME"].str.lower()
ni["NAME"] = ni["NAME"].str.capitalize()

In [3]:
# scotland and wales split unnecessarily into many polygons for the same region
# as such, I have merged them into one polygon per region
sct_wls = sct_wls.dissolve(by='NAME', aggfunc='sum')
sct_wls = sct_wls.reset_index()
sct_wls

,NAME,geometry,AREA_CODE,DESCRIPTIO,FILE_NAME,NUMBER,NUMBER0,POLYGON_ID,UNIT_ID,CODE,HECTARES,AREA,TYPE_CODE,DESCRIPT0,TYPE_COD0,DESCRIPT1
0,Central Scotland PER,"POLYGON ((273712.622 645542.835, 273719.020 64...",SPE,Scottish Parliament Electoral Region,CENTRAL_SCOTLAND_PER,1481,1481,124639,41546,S17000019,95483.869,1779.513,VA,CIVIL VOTING AREA,0,0
1,Glasgow PER,"POLYGON ((258096.702 672970.504, 258136.798 67...",SPE,Scottish Parliament Electoral Region,GLASGOW_PER,1482,1482,126812,41543,S17000020,20946.624,185.985,VA,CIVIL VOTING AREA,0,0
2,Highlands and Islands PER,"MULTIPOLYGON (((55661.497 780686.500, 55667.49...",SPESPESPESPESPESPESPESPESPESPESPESPESPESPESPES...,Scottish Parliament Electoral RegionScottish P...,HIGHLANDS_AND_ISLANDS_PERHIGHLANDS_AND_ISLANDS...,3544407,3544407,354159521,110426610,S17000011S17000011S17000011S17000011S17000011S...,4125655.196,91527.212,VAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVAVA...,CIVIL VOTING AREACIVIL VOTING AREACIVIL VOTING...,0,0
3,Lothian PER,"MULTIPOLYGON (((302334.395 655991.635, 302261....",SPE,Scottish Parliament Electoral Region,LOTHIAN_PER,1845,1845,127109,41548,S17000012,87283.885,1380.037,VA,CIVIL VOTING AREA,0,0
4,Mid Scotland and Fife PER,"MULTIPOLYGON (((329246.501 683248.502, 329259....",SPE,Scottish Parliament Electoral Region,MID_SCOTLAND_AND_FIFE_PER,1844,1844,124448,41547,S17000013,921251.932,9183.679,VA,CIVIL VOTING AREA,0,0
5,Mid and West Wales PER,"MULTIPOLYGON (((173935.497 205587.601, 173950....",WPE,Welsh Parliament Electoral Region,MID_AND_WEST_WALES_PER,1840,1840,125400,41534,W10000006,1327687.465,16492.446,VA,CIVIL VOTING AREA,0,0
6,North East Scotland PER,"MULTIPOLYGON (((349502.340 868854.560, 349490....",SPE,Scottish Parliament Electoral Region,NORTH_EAST_SCOTLAND_PER,1842,1842,126822,41542,S17000014,901018.611,6177.947,VA,CIVIL VOTING AREA,0,0
7,North Wales PER,"MULTIPOLYGON (((310816.300 385718.100, 310835....",WPE,Welsh Parliament Electoral Region,NORTH_WALES_PER,1841,1841,129136,41535,W10000001,417239.114,13517.146,VA,CIVIL VOTING AREA,0,0
8,South Scotland PER,"MULTIPOLYGON (((266029.102 543547.701, 266029....",SPE,Scottish Parliament Electoral Region,SOUTH_SCOTLAND_PER,1843,1843,127122,41544,S17000015,1638266.161,27126.801,VA,CIVIL VOTING AREA,0,0
9,South Wales Central PER,"MULTIPOLYGON (((322144.197 165168.900, 322164....",WPE,Welsh Parliament Electoral Region,SOUTH_WALES_CENTRAL_PER,1838,1838,122463,41532,W10000007,87667.662,1568.413,VA,CIVIL VOTING AREA,0,0


In [4]:
# convert the shapefiles of northern ireland to BNG
print(en.crs)
print(ni.crs)

ni = ni.to_crs(en.crs)

EPSG:27700
EPSG:29902


In [5]:
len(sct_wls)

13

In [6]:
UK = pd.concat([en[["NAME", "geometry"]], sct_wls[["NAME", "geometry"]]], ignore_index=True)
UK = pd.concat([UK, ni[["NAME", "geometry"]]], ignore_index=True)
               
UK = gpd.GeoDataFrame(UK)
UK.set_geometry("geometry", inplace=True)
UK

,NAME,geometry
0,East Midlands English Region,"POLYGON ((413673.997 400018.995, 413676.198 39..."
1,London English Region,"POLYGON ((531024.595 200933.623, 531026.320 20..."
2,North West English Region,"POLYGON ((374209.550 348536.660, 374212.450 34..."
3,West Midlands English Region,"POLYGON ((360408.943 340399.192, 360410.895 34..."
4,Yorkshire and the Humber English Region,"POLYGON ((424875.080 379990.440, 424872.170 37..."
5,South East English Region,"MULTIPOLYGON (((490173.603 256107.999, 490234...."
6,North East English Region,"MULTIPOLYGON (((398217.260 657393.560, 398226...."
7,Eastern English Region,"MULTIPOLYGON (((647445.460 323359.390, 647443...."
8,South West English Region,"MULTIPOLYGON (((418659.802 242642.204, 418662...."
9,Central Scotland PER,"POLYGON ((273712.622 645542.835, 273719.020 64..."


In [7]:
"""
SPARQL query to wikidata endpoint to get all MPs and their constituencies locations

"""
from SPARQLWrapper import SPARQLWrapper, JSON

const_df = pd.DataFrame()
sparql  = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?person ?const ?personLabel ?constLabel ?location
WHERE
{
  ?person p:P39 ?position.
  ?position ps:P39 wd:Q77685926.
  ?position pq:P768 ?const.
  ?const wdt:P625 ?location 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
""")

sparql.setReturnFormat(JSON)
result = sparql.query().convert()
# results -> bindings returns a list of dictionaries
result = result["results"]["bindings"]

# removed type from the results
for item in result:
    for key in item:
        item[key] = item[key]["value"]
const_df = pd.DataFrame(result)
const_df.head(3)

,const,person,location,personLabel,constLabel
0,http://www.wikidata.org/entity/Q280886,http://www.wikidata.org/entity/Q391526,Point(-2.504 51.462),Chris Skidmore,Kingswood
1,http://www.wikidata.org/entity/Q581626,http://www.wikidata.org/entity/Q19882694,Point(-3.892 55.845),Neil Gray,Airdrie and Shotts
2,http://www.wikidata.org/entity/Q588875,http://www.wikidata.org/entity/Q303883,Point(0.16 51.46),David Evennett,Bexleyheath and Crayford


In [8]:
const_df = const_df.drop_duplicates()
const_df = const_df.astype({"constLabel": "object", "personLabel": "object"})

In [9]:
# get the BNG location of each constituency
from bng_latlon import WGS84toOSGB36
const_df["easting"] = None
const_df["northing"] = None
for index, row in const_df.iterrows():
    lon, lat = row["location"].replace("Point(", "").replace(")", "").split(" ")

    lat = float(lat)
    lon = float(lon)

    x, y = WGS84toOSGB36(lat, lon)

    const_df.at[index, "easting"] = x
    const_df.at[index, "northing"] = y
const_df = const_df.drop(columns=["location"])

In [10]:
import shapely
import numpy as np

# checks which larger region each MPs constituency is in

# help from https://stackoverflow.com/questions/7861196/check-if-a-geopoint-with-latitude-and-longitude-is-within-a-shapefile



for index, row in const_df.iterrows():


    point = shapely.geometry.Point(row["easting"], row["northing"])
    point_contained = False

    for _, row2 in UK.iterrows():
        if row2["geometry"].contains(point):
            const_df.at[index, "region"] = row2["NAME"]
            point_contained = True
            break

    # some of the wikidata coordinates are in water
    # find the closest region to the point  
    if not point_contained:

        shortest_dist = np.inf
        point = shapely.geometry.Point(row["easting"], row["northing"])

        for _, row2 in UK.iterrows():
            dist = row2["geometry"].boundary.distance(point)

            if dist < shortest_dist:
                shortest_dist = dist
                closest_const = row2["NAME"]
        const_df.at[index, "region"] = closest_const

In [11]:
# iterate through each question, link it to the MP who asked it, and the region they are from
questions["region"] = None
for index, row in questions.iterrows():
    # find the row in const_df with the same name as the MP who asked the question

    row2 = const_df[const_df["person"] == row["wikidataperson"]]

    region = row2["region"].values[0]
    # set the region of the question to that region
    questions.at[index, "region"] = region

In [12]:
questions["region"].value_counts()

region
London English Region                      5736
Yorkshire and the Humber English Region    4672
North West English Region                  3296
South East English Region                  2604
West Midlands English Region               2137
North East English Region                  1798
South West English Region                  1476
East Midlands English Region               1017
Eastern English Region                      570
Glasgow PER                                 557
Down                                        531
South Wales West PER                        355
South Wales East PER                        279
Mid and West Wales PER                      273
South Scotland PER                          258
Central Scotland PER                        243
Highlands and Islands PER                   187
Londonderry                                 160
West Scotland PER                           159
South Wales Central PER                     137
Lothian PER                      

In [13]:
# go through each region and combine all the questions asked by MPs from that region into one string
corpus = {}
for region in questions["region"].unique():
    combined_text = ""
    for index, row in questions[questions["region"] == region].iterrows():
        combined_text += row["text"] + " "
    
    corpus[region] = combined_text


In [14]:
for key in corpus:
    print(key)

East Midlands English Region
London English Region
South East English Region
Yorkshire and the Humber English Region
North West English Region
West Midlands English Region
South Wales West PER
Mid and West Wales PER
North East English Region
West Scotland PER
South Wales East PER
Glasgow PER
South West English Region
Eastern English Region
North Wales PER
South Scotland PER
Londonderry
Antrim
Highlands and Islands PER
Central Scotland PER
Lothian PER
North East Scotland PER
South Wales Central PER
Down
Mid Scotland and Fife PER


In [15]:
"""Code from here http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html"""

import re
from nltk.stem.porter import PorterStemmer
from stop_words import get_stop_words
from nltk.tokenize import RegexpTokenizer

def text_to_tokens(doc):
    p_stemmer = PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')

    doc = doc.lower()

    # remove any letter that is of the form (a)
    doc = re.sub(r'\(\w\)', '', doc)
    


    # convert to tokens
    tokens = tokenizer.tokenize(doc) 
    # remove stop words from tokens
    en_stop = get_stop_words('en')
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # remove additional words that are not in the stop words list
    additional_stop_words = ["(a)","secretary", "state", "ask", "asking", "department", "departments", "minister", "ministers", "government", "s", "i", "ii"]
    stopped_tokens = [i for i in stopped_tokens if not i in additional_stop_words]

    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]

    return stemmed_tokens

In [16]:
# tokenize and look at the most common words in each region
from nltk import FreqDist

#corpus["Down"]
tokens = text_to_tokens(corpus["Glasgow PER"])
tokens = FreqDist(tokens)
tokens.most_common(50)

corpus["Glasgow PER"]


"To ask the Secretary of State for Environment, Food and Rural Affairs, whether she has had discussions with the RCVS Veterinary Nurses Council on changing the Veterinary Surgeons Act 1966 to (a) replace the criminal standard of proof with the civil standard and (b) make improvements to the Fitness to Practise disciplinary system. To ask the Secretary of State for Business, Energy and Industrial Strategy, whether specific financial support for people in off gas grid homes will be available in 2023/24. To ask the Secretary of State for Business, Energy and Industrial Strategy, whether additional support payments for those living off gas grid will be  available to those (a) on Economy 7 tariffs and (b) using electric only heating systems. To ask the Secretary of State for Environment, Food and Rural Affairs, what recent assessment she has made of the potential impact of the effect of inflation in the cost of pet food for domestic animals on trends in the level of animal welfare. To ask t

In [17]:
list(corpus.keys())

['East Midlands English Region',
 'London English Region',
 'South East English Region',
 'Yorkshire and the Humber English Region',
 'North West English Region',
 'West Midlands English Region',
 'South Wales West PER',
 'Mid and West Wales PER',
 'North East English Region',
 'West Scotland PER',
 'South Wales East PER',
 'Glasgow PER',
 'South West English Region',
 'Eastern English Region',
 'North Wales PER',
 'South Scotland PER',
 'Londonderry',
 'Antrim',
 'Highlands and Islands PER',
 'Central Scotland PER',
 'Lothian PER',
 'North East Scotland PER',
 'South Wales Central PER',
 'Down',
 'Mid Scotland and Fife PER']

In [18]:
"""Code from here http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html"""

# 
doc_set = []

# put documents into a list in order of the keys in the corpus dictionary
for key in corpus.keys():
    doc_set = doc_set + [corpus[key]]


texts = []
for doc in doc_set:
    
    # add tokens to list
    texts.append(text_to_tokens(doc))

In [19]:
print(doc_set[0])

Which three departments had the lowest proportion of staff attending the office in January 2023. To ask the Secretary of State for Education, what steps her Department is taking to help support sixth-form colleges with rises in the cost of energy. To ask the Secretary of State for Health and Social Care, whether he has a timetable for the introduction of legislative proposals to give care home residents a legal right to receive visitors. To ask the Secretary of State for Education, what steps her Department is taking to help support sixth-form colleges with costs associated with increases in the level of inflation. To ask the Secretary of State for Education, how many further education college infrastructure projects are affected by the reclassification of colleges. To ask the Secretary of State for Environment, Food and Rural Affairs, whether she has plans to phase out the caging of laying hens by 2027. To ask the Secretary of State for Work and Pensions, how many people have accessed

In [20]:
from gensim import corpora

# create dictionary which contains the number of times a word appears and other statistics
dictionary = corpora.Dictionary(texts)


# makes a list for each document with vectors of (token_id, token_count)
bow_vectors = [dictionary.doc2bow(text) for text in texts]

In [21]:
# applying LDA model to the bow_vectors
import gensim.models.ldamodel


ldamodel = gensim.models.ldamodel.LdaModel(bow_vectors, num_topics=4, id2word = dictionary, passes=200)

In [22]:
# look at coherence 
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
cohernece_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', cohernece_lda)


Coherence Score:  0.311138584977691


In [41]:
print(ldamodel.print_topics(num_topics=4, num_words=2))

[(0, '0.025*"ireland" + 0.023*"northern"'), (1, '0.021*"school" + 0.018*"educ"'), (2, '0.014*"health" + 0.013*"care"'), (3, '0.014*"assess" + 0.014*"health"')]


In [44]:
regional_topics = pd.DataFrame(columns=["region", "topic", "topic_prob"])

# bow vector is ordered the same as the corpus keys
names = list(corpus.keys())

for index, vector in enumerate(bow_vectors):


    doc_topics = ldamodel.get_document_topics(vector)

    for topic in doc_topics:
        regional_topics = pd.concat(
            [regional_topics, 
             pd.DataFrame({"region": [names[index]], "topic": [topic[0]], "topic_prob": [topic[1]]})], 
             ignore_index=True)
        

C:\Users\josep\AppData\Local\Temp\ipykernel_22736\1967087118.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  regional_topics = pd.concat(


In [49]:
regional_topics[regional_topics["topic"] == 3].sort_values(by="topic_prob", ascending=False)

,region,topic,topic_prob
5,Yorkshire and the Humber English Region,3,0.982996
9,West Midlands English Region,3,0.929746
20,South Wales East PER,3,0.833910
3,South East English Region,3,0.776152
32,South Scotland PER,3,0.747099
56,Mid Scotland and Fife PER,3,0.688288
22,Glasgow PER,3,0.561048
24,South West English Region,3,0.507397
54,Down,3,0.419525
27,Eastern English Region,3,0.409814
